In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "5"
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
import Levenshtein
import cv2
from PIL import Image
from matplotlib import pyplot as plt
import seaborn as sns
import time
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from torch.nn.utils.rnn import pack_padded_sequence
import torchvision
import PIL.Image as Image
from torchvision.transforms import ToTensor, ToPILImage
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch
from torch import nn
from nltk.translate.bleu_score import corpus_bleu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

<h3>Put the image paths and caption paths into lists to facilitate their usage during the loading and processing of the dataset, especially for handling both train and test sets.</h3>

In [ ]:
class_id = []
with os.scandir('../input/oxford-102flowers/data/train') as entries:
    for entry in entries:
        class_id.append(int(entry.name))

list_caption = []
list_image = []
train_list_image = []
train_class_id =[]
test_list_image = []
train_caption = []
text_path = "../input/text-description/text_arab/"
trian_path = "../input/oxford-102flowers/data/train"
with os.scandir('../input/oxford-102flowers/data/train') as entries:
    for entry in entries:
        d=os.path.join(trian_path,entry.name)
        with os.scandir(d) as img:
            for m in img:
                text = m.name[:-4]
                text = text +".txt"
                image_train_path=os.path.join(d,m.name)
                train_list_image.append(image_train_path)
                train_class_id.append(int(entry.name))
                train_caption.append(text_path+text)
valid_path = "../input/oxford-102flowers/data/valid"
with os.scandir('../input/oxford-102flowers/data/valid') as entries:
    for entry in entries:
        d=os.path.join(valid_path,entry.name)
        with os.scandir(d) as img:
            for m in img:
                text = m.name[:-4]
                text = text +".txt"
                image_train_path=os.path.join(d,m.name)
                train_list_image.append(image_train_path)
                train_class_id.append(int(entry.name))
                train_caption.append(text_path+text)
                
test_caption = []
test_list_image = []
test_class_id =[]
test_path = "../input/oxford-102flowers/data/test"
with os.scandir('../input/oxford-102flowers/data/test') as entries:
    for entry in entries:
        d=os.path.join(test_path,entry.name)
        with os.scandir(d) as img:
            for m in img:
                text = m.name[:-4]
                text = text +".txt"
                image_train_path=os.path.join(d,m.name)
                test_list_image.append(image_train_path)
                test_class_id.append(int(entry.name))
                test_caption.append(text_path + text)

In [ ]:

print(train_list_image[:5])
print(train_class_id[:5])
print(train_caption[:5])
print(test_list_image[:5])
print(test_class_id[:5])
print(test_caption[:5])

In [ ]:
print(len(train_list_image))
print(len(train_class_id))
print(len(train_caption))
print(len(test_list_image))
print(len(test_class_id))
print(len(test_caption))

In [ ]:
!pip install pyarabic
import pyarabic.araby as araby
import pyarabic.number as number

<h3>Clean and toknize the Arabic text</h3>

In [ ]:
tokens_new = []
captions_paths = train_caption + test_caption
for i in captions_paths:
    with open(i, "r") as f:
        captions = f.read().split('\n')
    for cap in captions:
        cap = araby.strip_tashkeel(cap)
        cap = araby.strip_lastharaka(cap)
        cap = araby.normalize_hamza(cap)
        cap = araby.tokenize(cap)
        for s in cap:
            if not s.startswith('و') and not s.startswith('.') and not s.startswith('�') and not s.startswith(','):
                if s not in tokens_new:
                    tokens_new.append(s)
print(len(tokens_new))

In [ ]:
def line_to_sequence(caption_path):
    bag_word = []
    with open(caption_path, "r") as f:
        captions = f.read().split('\n')
    for cap in captions:
        cap = araby.strip_tashkeel(cap)
        cap = araby.strip_lastharaka(cap)
        cap = araby.normalize_hamza(cap)
        cap = araby.tokenize(cap)
        for s in cap:
            if not s.startswith('و') and not s.startswith('.') and not s.startswith('�') and not s.startswith(','):
                bag_word.append(s)
    return bag_word
    

In [ ]:
print(len(tokens_new))
print(tokens_new[:100])


<h3>Add the End and Strat for each caption.</h3>

In [ ]:
        ixtoword = {}
        ixtoword[0] = '<end>'
        ixtoword[1] = '<out>'
        wordtoix = {}
        wordtoix['<end>'] = 0
        wordtoix['<out>'] = 1
        ix = 2
        for w in tokens_new:
            wordtoix[w] = ix
            ixtoword[ix] = w
            ix += 1

In [ ]:

class tokinazer():
    def __init__(self,wordtoix,ixtoword):
        super(tokinazer, self).__init__()
        self.wordtoix = wordtoix
        self.ixtoword = ixtoword
        
    def encode(self,seq):
        return [self.wordtoix[i] for i in seq]
    def decode(self,seq):
        return [self.ixtoword[i] for i in seq]
tokinazer = tokinazer(wordtoix,ixtoword)

<h3>Determine the maximum text length and use it as the sequences length.</h3>

In [ ]:
s = train_caption + test_caption
max_seq = 0
for i in s:
    seq = line_to_sequence(i)
    seq = tokinazer.encode(seq)
    if max_seq < len(seq):
        max_seq = len(seq)

In [ ]:
max_seq

<h3>Pre-process of the iamges and thier coptions</h3>

In [ ]:
def text_to_tensor(path,max_len):
    seq = line_to_sequence(path)
    seq = tokinazer.encode(seq)
    seq.append(0)
    len_seq = len(seq)
    tensor = torch.LongTensor(F.pad(torch.LongTensor(seq), pad=(0, (max_len+1 )- len_seq) , mode='constant', value=0)) 
    return tensor,len_seq



def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')
    
        
train_transforms =  transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #transforms.Normalize([0.5], [0.5]),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

class dataset(torch.utils.data.Dataset):

    def __init__(self,images_paths,captions_paths,max_len,class_id,transform=None):
        self.images_paths = images_paths
        self.captions_paths = captions_paths
        self.max_len = max_len 
        self.class_id = class_id
        self.transform = transform 
         
    #dataset length
    def __len__(self):
        self.filelength = len(self.images_paths)
        return self.filelength
    
    #load an one of images
    def __getitem__(self,idx):
     #   img_path = self.file_list[idx]
        img = pil_loader(self.images_paths[idx].strip())
        tensor_caption,len_seq = text_to_tensor(self.captions_paths[idx],self.max_len)
        #seq_tensor = Variable(torch.zeros((len(vectorized_seqs), seq_lengths.max()))).long(
        img_transformed = self.transform(img)
        id_ = self.class_id[idx]
        #img_transformed = img_transformed.expand(3,256,256)
        return img_transformed,tensor_caption,len_seq,id_

In [ ]:
print(len(train_list_image))
print(len(train_class_id))
print(len(train_caption))
print(len(test_list_image))
print(len(test_class_id))
print(len(test_caption))

In [ ]:
images_paths  = train_list_image[:7168]
captions_paths = train_caption[:7168]
class_id_train = train_class_id[:7168] 
test_images = test_list_image[:992] 
test_captions = test_caption[:992]
class_id_test = test_class_id[:992]
train_data = dataset(images_paths,captions_paths,max_seq,class_id_train,transform=train_transforms)
train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size=32, shuffle=True)
test_data = dataset(test_images,test_captions,max_seq,class_id_test,transform=train_transforms)
test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size=32, shuffle=False)


In [ ]:
print(len(train_data))
print(len(train_loader))
print(len(test_data))
print(len(test_loader))

In [ ]:
print(device)

<h3>Define the text Encoder to produce the word and sentence Embedding</h3>

In [ ]:
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
class RNN_ENCODER(nn.Module):
    def __init__(self, ntoken, ninput=300, drop_prob=0.5,
                 nhidden=128, nlayers=1, bidirectional=True):
        super(RNN_ENCODER, self).__init__()
        self.n_steps = 18
        self.ntoken = ntoken  # size of the dictionary
        self.ninput = ninput  # size of each embedding vector
        self.drop_prob = drop_prob  # probability of an element to be zeroed
        self.nlayers = nlayers  # Number of recurrent layers
        self.bidirectional = bidirectional
        self.rnn_type = 'LSTM'
        if bidirectional:
            self.num_directions = 2
        else:
            self.num_directions = 1
        # number of features in the hidden state
        self.nhidden = nhidden // self.num_directions

        self.define_module()
        self.init_weights()

    def define_module(self):
        self.encoder = nn.Embedding(self.ntoken, self.ninput)
        self.drop = nn.Dropout(self.drop_prob)
        if self.rnn_type == 'LSTM':
            # dropout: If non-zero, introduces a dropout layer on
            # the outputs of each RNN layer except the last layer
            self.rnn = nn.LSTM(self.ninput, self.nhidden,
                               self.nlayers, batch_first=True,
                               dropout=self.drop_prob,
                               bidirectional=self.bidirectional)
        elif self.rnn_type == 'GRU':
            self.rnn = nn.GRU(self.ninput, self.nhidden,
                              self.nlayers, batch_first=True,
                              dropout=self.drop_prob,
                              bidirectional=self.bidirectional)
        else:
            raise NotImplementedError

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        # Do not need to initialize RNN parameters, which have been initialized
        # http://pytorch.org/docs/master/_modules/torch/nn/modules/rnn.html#LSTM
        # self.decoder.weight.data.uniform_(-initrange, initrange)
        # self.decoder.bias.data.fill_(0)

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (Variable(weight.new(self.nlayers * self.num_directions,
                                        bsz, self.nhidden).zero_()),
                    Variable(weight.new(self.nlayers * self.num_directions,
                                        bsz, self.nhidden).zero_()))
        else:
            return Variable(weight.new(self.nlayers * self.num_directions,
                                       bsz, self.nhidden).zero_())

    def forward(self, captions, cap_lens, hidden, mask=None):
        # input: torch.LongTensor of size batch x n_steps
        # --> emb: batch x n_steps x ninput
        emb = self.drop(self.encoder(captions))
        #
        # Returns: a PackedSequence object
        cap_lens = cap_lens.data.tolist()
        emb = pack_padded_sequence(emb, cap_lens, batch_first=True)
        # #hidden and memory (num_layers * num_directions, batch, hidden_size):
        # tensor containing the initial hidden state for each element in batch.
        # #output (batch, seq_len, hidden_size * num_directions)
        # #or a PackedSequence object:
        # tensor containing output features (h_t) from the last layer of RNN
        output, hidden = self.rnn(emb, hidden)
        # PackedSequence object
        # --> (batch, seq_len, hidden_size * num_directions)
        output = pad_packed_sequence(output, batch_first=True)[0]
        # output = self.drop(output)
        # --> batch x hidden_size*num_directions x seq_len
        words_emb = output.transpose(1, 2)
        # --> batch x num_directions*hidden_size
        if self.rnn_type == 'LSTM':
            sent_emb = hidden[0].transpose(0, 1).contiguous()
        else:
            sent_emb = hidden.transpose(0, 1).contiguous()
        sent_emb = sent_emb.view(-1, self.nhidden * self.num_directions)
        return words_emb, sent_emb

In [ ]:
text_encoder = RNN_ENCODER(len(tokens_new), nhidden=256)

<h3>Define the model of Enhanced DF-GAN.</h3>

In [ ]:
import queue
import collections
import threading

class SyncMaster(object):
    """An abstract `SyncMaster` object.
    - During the replication, as the data parallel will trigger an callback of each module, all slave devices should
    call `register(id)` and obtain an `SlavePipe` to communicate with the master.
    - During the forward pass, master device invokes `run_master`, all messages from slave devices will be collected,
    and passed to a registered callback.
    - After receiving the messages, the master device should gather the information and determine to message passed
    back to each slave devices.
    """

    def __init__(self, master_callback):
        """
        Args:
            master_callback: a callback to be invoked after having collected messages from slave devices.
        """
        self._master_callback = master_callback
        self._queue = queue.Queue()
        self._registry = collections.OrderedDict()
        self._activated = False

    def __getstate__(self):
        return {'master_callback': self._master_callback}

    def __setstate__(self, state):
        self.__init__(state['master_callback'])

    def register_slave(self, identifier):
        """
        Register an slave device.
        Args:
            identifier: an identifier, usually is the device id.
        Returns: a `SlavePipe` object which can be used to communicate with the master device.
        """
        if self._activated:
            assert self._queue.empty(), 'Queue is not clean before next initialization.'
            self._activated = False
            self._registry.clear()
        future = FutureResult()
        self._registry[identifier] = _MasterRegistry(future)
        return SlavePipe(identifier, self._queue, future)

    def run_master(self, master_msg):
        """
        Main entry for the master device in each forward pass.
        The messages were first collected from each devices (including the master device), and then
        an callback will be invoked to compute the message to be sent back to each devices
        (including the master device).
        Args:
            master_msg: the message that the master want to send to itself. This will be placed as the first
            message when calling `master_callback`. For detailed usage, see `_SynchronizedBatchNorm` for an example.
        Returns: the message to be sent back to the master device.
        """
        self._activated = True

        intermediates = [(0, master_msg)]
        for i in range(self.nr_slaves):
            intermediates.append(self._queue.get())

        results = self._master_callback(intermediates)
        assert results[0][0] == 0, 'The first result should belongs to the master.'

        for i, res in results:
            if i == 0:
                continue
            self._registry[i].result.put(res)

        for i in range(self.nr_slaves):
            assert self._queue.get() is True

        return results[0][1]

    @property
    def nr_slaves(self):
        return len(self._registry)
    

In [ ]:
import collections

import torch
import torch.nn.functional as F

from torch.nn.modules.batchnorm import _BatchNorm
from torch.nn.parallel._functions import ReduceAddCoalesced, Broadcast

__all__ = ['SynchronizedBatchNorm1d', 'SynchronizedBatchNorm2d', 'SynchronizedBatchNorm3d']


def _sum_ft(tensor):
    """sum over the first and last dimention"""
    return tensor.sum(dim=0).sum(dim=-1)


def _unsqueeze_ft(tensor):
    """add new dementions at the front and the tail"""
    return tensor.unsqueeze(0).unsqueeze(-1)


_ChildMessage = collections.namedtuple('_ChildMessage', ['sum', 'ssum', 'sum_size'])
_MasterMessage = collections.namedtuple('_MasterMessage', ['sum', 'inv_std'])


class _SynchronizedBatchNorm(_BatchNorm):
    def __init__(self, num_features, eps=1e-5, momentum=0.1, affine=True):
        super(_SynchronizedBatchNorm, self).__init__(num_features, eps=eps, momentum=momentum, affine=affine)

        self._sync_master = SyncMaster(self._data_parallel_master)

        self._is_parallel = False
        self._parallel_id = None
        self._slave_pipe = None

    def forward(self, input):
        # If it is not parallel computation or is in evaluation mode, use PyTorch's implementation.
        if not (self._is_parallel and self.training):
            return F.batch_norm(
                input, self.running_mean, self.running_var, self.weight, self.bias,
                self.training, self.momentum, self.eps)

        # Resize the input to (B, C, -1).
        input_shape = input.size()
        input = input.view(input.size(0), self.num_features, -1)

        # Compute the sum and square-sum.
        sum_size = input.size(0) * input.size(2)
        input_sum = _sum_ft(input)
        input_ssum = _sum_ft(input ** 2)

        # Reduce-and-broadcast the statistics.
        if self._parallel_id == 0:
            mean, inv_std = self._sync_master.run_master(_ChildMessage(input_sum, input_ssum, sum_size))
        else:
            mean, inv_std = self._slave_pipe.run_slave(_ChildMessage(input_sum, input_ssum, sum_size))

        # Compute the output.
        if self.affine:
            # MJY:: Fuse the multiplication for speed.
            output = (input - _unsqueeze_ft(mean)) * _unsqueeze_ft(inv_std * self.weight) + _unsqueeze_ft(self.bias)
        else:
            output = (input - _unsqueeze_ft(mean)) * _unsqueeze_ft(inv_std)

        # Reshape it.
        return output.view(input_shape)

    def __data_parallel_replicate__(self, ctx, copy_id):
        self._is_parallel = True
        self._parallel_id = copy_id

        # parallel_id == 0 means master device.
        if self._parallel_id == 0:
            ctx.sync_master = self._sync_master
        else:
            self._slave_pipe = ctx.sync_master.register_slave(copy_id)

    def _data_parallel_master(self, intermediates):
        """Reduce the sum and square-sum, compute the statistics, and broadcast it."""

        # Always using same "device order" makes the ReduceAdd operation faster.
        # Thanks to:: Tete Xiao (http://tetexiao.com/)
        intermediates = sorted(intermediates, key=lambda i: i[1].sum.get_device())

        to_reduce = [i[1][:2] for i in intermediates]
        to_reduce = [j for i in to_reduce for j in i]  # flatten
        target_gpus = [i[1].sum.get_device() for i in intermediates]

        sum_size = sum([i[1].sum_size for i in intermediates])
        sum_, ssum = ReduceAddCoalesced.apply(target_gpus[0], 2, *to_reduce)
        mean, inv_std = self._compute_mean_std(sum_, ssum, sum_size)

        broadcasted = Broadcast.apply(target_gpus, mean, inv_std)

        outputs = []
        for i, rec in enumerate(intermediates):
            outputs.append((rec[0], _MasterMessage(*broadcasted[i*2:i*2+2])))

        return outputs

    def _compute_mean_std(self, sum_, ssum, size):
        """Compute the mean and standard-deviation with sum and square-sum. This method
        also maintains the moving average on the master device."""
        assert size > 1, 'BatchNorm computes unbiased standard-deviation, which requires size > 1.'
        mean = sum_ / size
        sumvar = ssum - sum_ * mean
        unbias_var = sumvar / (size - 1)
        bias_var = sumvar / size

        self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * mean.data
        self.running_var = (1 - self.momentum) * self.running_var + self.momentum * unbias_var.data

        return mean, bias_var.clamp(self.eps) ** -0.5




class SynchronizedBatchNorm2d(_SynchronizedBatchNorm):
    def _check_input_dim(self, input):
        if input.dim() != 4:
            raise ValueError('expected 4D input (got {}D input)'
                             .format(input.dim()))
        super(SynchronizedBatchNorm2d, self)._check_input_dim(input)

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from collections import OrderedDict
#from sync_batchnorm import SynchronizedBatchNorm2d

BatchNorm = SynchronizedBatchNorm2d


class NetG(nn.Module):
    def __init__(self, ngf=64, nz=100):
        super(NetG, self).__init__()
        self.ngf = ngf

        self.conv_mask = nn.Sequential(nn.Conv2d(8 * ngf, 100, 3, 1, 1),
                                       BatchNorm(100),
                                       nn.ReLU(),
                                       nn.Conv2d(100, 1, 1, 1, 0))

        # layer1输入的是一个100x1x1的随机噪声, 输出尺寸(ngf*8)x4x4
        self.fc = nn.Linear(nz, ngf * 8 * 4 * 4)
        self.block0 = G_Block(ngf * 8, ngf * 8)  # 4x4
        self.block1 = G_Block(ngf * 8, ngf * 8)  # 8x8
        self.block2 = G_Block(ngf * 8, ngf * 8)  # 16x16
        self.block3 = G_Block(ngf * 8, ngf * 8)  # 32x32
        self.block4 = G_Block(ngf * 8, ngf * 4)  # 64x64
        self.block5 = G_Block(ngf * 4, ngf * 2)  # 128x128
        self.block6 = G_Block(ngf * 2, ngf * 1, predict_mask=False)  # 256x256

        self.conv_img = nn.Sequential(
            BatchNorm(ngf),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ngf, 3, 3, 1, 1),
            nn.Tanh(),
        )

    def forward(self, x, c):

        out = self.fc(x)
        out = out.view(x.size(0), 8 * self.ngf, 4, 4)
        hh, ww = out.size(2), out.size(3)
        stage_mask = self.conv_mask(out)
        fusion_mask = torch.sigmoid(stage_mask)
        stage_mask_4 = fusion_mask
        out, stage_mask = self.block0(out, c, fusion_mask)

        out = F.interpolate(out, scale_factor=2)
        hh, ww = out.size(2), out.size(3)
        stage_mask = F.interpolate(stage_mask, size=(hh, ww), mode='bilinear', align_corners=True)
        fusion_mask = torch.sigmoid(stage_mask)
        stage_mask_8 = fusion_mask
        out, stage_mask = self.block1(out, c, fusion_mask)

        out = F.interpolate(out, scale_factor=2)
        hh, ww = out.size(2), out.size(3)
        stage_mask = F.interpolate(stage_mask, size=(hh, ww), mode='bilinear', align_corners=True)
        fusion_mask = torch.sigmoid(stage_mask)
        stage_mask_16 = fusion_mask
        out, stage_mask = self.block2(out, c, fusion_mask)

        out = F.interpolate(out, scale_factor=2)
        hh, ww = out.size(2), out.size(3)
        stage_mask = F.interpolate(stage_mask, size=(hh, ww), mode='bilinear', align_corners=True)
        fusion_mask = torch.sigmoid(stage_mask)
        stage_mask_32 = fusion_mask
        out, stage_mask = self.block3(out, c, fusion_mask)

        out = F.interpolate(out, scale_factor=2)
        hh, ww = out.size(2), out.size(3)
        stage_mask = F.interpolate(stage_mask, size=(hh, ww), mode='bilinear', align_corners=True)
        fusion_mask = torch.sigmoid(stage_mask)
        stage_mask_64 = fusion_mask
        out, stage_mask = self.block4(out, c, fusion_mask)

        out = F.interpolate(out, scale_factor=2)
        hh, ww = out.size(2), out.size(3)
        stage_mask = F.interpolate(stage_mask, size=(hh, ww), mode='bilinear', align_corners=True)
        fusion_mask = torch.sigmoid(stage_mask)
        stage_mask_128 = fusion_mask
        out, stage_mask = self.block5(out, c, fusion_mask)

        out = F.interpolate(out, scale_factor=2)
        hh, ww = out.size(2), out.size(3)
        stage_mask = F.interpolate(stage_mask, size=(hh, ww), mode='bilinear', align_corners=True)
        fusion_mask = torch.sigmoid(stage_mask)
        stage_mask_256 = fusion_mask
        out, _ = self.block6(out, c, fusion_mask)

        out = self.conv_img(out)

        # return out, fusion_mask
        return out, [stage_mask_4, stage_mask_8, stage_mask_16, stage_mask_32,
                     stage_mask_64, stage_mask_128, stage_mask_256]


class G_Block(nn.Module):

    def __init__(self, in_ch, out_ch, num_w=256, predict_mask=True):
        super(G_Block, self).__init__()

        self.learnable_sc = in_ch != out_ch
        self.predict_mask = predict_mask
        self.c1 = nn.Conv2d(in_ch, out_ch, 3, 1, 1)
        self.c2 = nn.Conv2d(out_ch, out_ch, 3, 1, 1)
        self.affine0 = affine(in_ch)
        #self.affine1 = affine(in_ch)
        self.affine2 = affine(out_ch)
        #self.affine3 = affine(out_ch)
        self.gamma = nn.Parameter(torch.zeros(1))
        if self.learnable_sc:
            self.c_sc = nn.Conv2d(in_ch, out_ch, 1, stride=1, padding=0)

        if self.predict_mask:
            self.conv_mask = nn.Sequential(nn.Conv2d(out_ch, 100, 3, 1, 1),
                                           BatchNorm(100),
                                           nn.ReLU(),
                                           nn.Conv2d(100, 1, 1, 1, 0))

    def forward(self, x, y=None, fusion_mask=None):
        out = self.shortcut(x) + self.gamma * self.residual(x, y, fusion_mask)

        if self.predict_mask:
            mask = self.conv_mask(out)
        else:
            mask = None

        return out, mask

    def shortcut(self, x):
        if self.learnable_sc:
            x = self.c_sc(x)
        return x

    def residual(self, x, y=None, fusion_mask=None):
        h = self.affine0(x, y, fusion_mask)
        h = nn.ReLU(inplace=True)(h)
        h = self.c1(h)

        h = self.affine2(h, y, fusion_mask)
        h = nn.ReLU(inplace=True)(h)
        return self.c2(h)


class affine(nn.Module):

    def __init__(self, num_features):
        super(affine, self).__init__()

        self.batch_norm2d = BatchNorm(num_features, affine=False)

        self.fc_gamma = nn.Sequential(OrderedDict([
            ('linear1', nn.Linear(256, 256)),
            ('relu1', nn.ReLU(inplace=True)),
            ('linear2', nn.Linear(256, num_features)),
        ]))
        self.fc_beta = nn.Sequential(OrderedDict([
            ('linear1', nn.Linear(256, 256)),
            ('relu1', nn.ReLU(inplace=True)),
            ('linear2', nn.Linear(256, num_features)),
        ]))
        self._initialize()

    def _initialize(self):
        nn.init.zeros_(self.fc_gamma.linear2.weight.data)
        nn.init.zeros_(self.fc_gamma.linear2.bias.data)
        nn.init.zeros_(self.fc_beta.linear2.weight.data)
        nn.init.zeros_(self.fc_beta.linear2.bias.data)

    def forward(self, x, y=None, fusion_mask=None):
        x = self.batch_norm2d(x)
        weight = self.fc_gamma(y)
        bias = self.fc_beta(y)

        if weight.dim() == 1:
            weight = weight.unsqueeze(0)
        if bias.dim() == 1:
            bias = bias.unsqueeze(0)

        size = x.size()
        weight = weight.unsqueeze(-1).unsqueeze(-1).expand(size)
        bias = bias.unsqueeze(-1).unsqueeze(-1).expand(size)
        weight = weight * fusion_mask + 1
        bias = bias * fusion_mask
        return weight * x + bias


class D_GET_LOGITS(nn.Module):
    def __init__(self, ndf):
        super(D_GET_LOGITS, self).__init__()
        self.df_dim = ndf

        self.joint_conv = nn.Sequential(
            nn.Conv2d(ndf * 16 + 256, ndf * 2, 3, 1, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, 1, 4, 1, 0, bias=False),
        )

    def forward(self, out, y):

        y = y.view(-1, 256, 1, 1)
        y = y.repeat(1, 1, 4, 4)
        h_c_code = torch.cat((out, y), 1)
        out = self.joint_conv(h_c_code)
        return out


# 定义鉴别器网络D
class NetD(nn.Module):
    def __init__(self, ndf):
        super(NetD, self).__init__()

        self.conv_img = nn.Conv2d(3, ndf, 3, 1, 1)  # 128
        self.block0 = resD(ndf * 1, ndf * 2)  # 64
        self.block1 = resD(ndf * 2, ndf * 4)  # 32
        self.block2 = resD(ndf * 4, ndf * 8)  # 16
        self.block3 = resD(ndf * 8, ndf * 16)  # 8
        self.block4 = resD(ndf * 16, ndf * 16)  # 4
        self.block5 = resD(ndf * 16, ndf * 16)  # 4

        self.COND_DNET = D_GET_LOGITS(ndf)

    def forward(self, x):

        out = self.conv_img(x)
        out = self.block0(out)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.block4(out)
        out = self.block5(out)

        return out


class resD(nn.Module):
    def __init__(self, fin, fout, downsample=True):
        super().__init__()
        self.downsample = downsample
        self.learned_shortcut = (fin != fout)
        self.conv_r = nn.Sequential(
            nn.Conv2d(fin, fout, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(fout, fout, 3, 1, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
        )

        self.conv_s = nn.Conv2d(fin, fout, 1, stride=1, padding=0)
        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self, x, c=None):
        return self.shortcut(x) + self.gamma * self.residual(x)

    def shortcut(self, x):
        if self.learned_shortcut:
            x = self.conv_s(x)
        if self.downsample:
            return F.avg_pool2d(x, 2)
        return x

    def residual(self, x):
        return self.conv_r(x)


def conv2d(in_feat, out_feat, kernel_size=3, stride=1, padding=1, bias=True, spectral_norm=False):
    conv = nn.Conv2d(in_feat, out_feat, kernel_size, stride, padding, bias=bias)
    if spectral_norm:
        return nn.utils.spectral_norm(conv, eps=1e-4)
    else:
        return conv


def linear(in_feat, out_feat, bias=True, spectral_norm=False):
    lin = nn.Linear(in_feat, out_feat, bias=bias)
    if spectral_norm:
        return nn.utils.spectral_norm(lin)
    else:
        return lin

In [ ]:
def prepare_labels(batch_size):
    # Kai: real_labels and fake_labels have data type: torch.float32
    # match_labels has data type: torch.int64
    real_labels = Variable(torch.FloatTensor(batch_size).fill_(1))
    fake_labels = Variable(torch.FloatTensor(batch_size).fill_(0))
    match_labels = Variable(torch.LongTensor(range(batch_size)))
    real_labels = real_labels.cuda()
    fake_labels = fake_labels.cuda()
    match_labels = match_labels.cuda()
    return real_labels, fake_labels, match_labels

In [ ]:
from torchvision import models
def conv1x1(in_planes, out_planes, bias=False):
    "1x1 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1,
                     padding=0, bias=bias)

def func_attention(query, context, gamma1):
    """
    query: batch x ndf x queryL
    context: batch x ndf x ih x iw (sourceL=ihxiw)
    mask: batch_size x sourceL
    """
    batch_size, queryL = query.size(0), query.size(2)
    ih, iw = context.size(2), context.size(3)
    sourceL = ih * iw

    # --> batch x sourceL x ndf
    context = context.view(batch_size, -1, sourceL)
    contextT = torch.transpose(context, 1, 2).contiguous()

    # Get attention
    # (batch x sourceL x ndf)(batch x ndf x queryL)
    # -->batch x sourceL x queryL
    attn = torch.bmm(contextT, query)  # Eq. (7) in AttnGAN paper
    # --> batch*sourceL x queryL
    attn = attn.cuda()
    attn = attn.view(batch_size * sourceL, queryL)
    attn = nn.Softmax()(attn)  # Eq. (8)

    # --> batch x sourceL x queryL
    attn = attn.view(batch_size, sourceL, queryL)
    # --> batch*queryL x sourceL
    attn = torch.transpose(attn, 1, 2).contiguous()
    attn = attn.view(batch_size * queryL, sourceL)
    #  Eq. (9)
    attn = attn * gamma1
    attn = nn.Softmax()(attn)
    attn = attn.view(batch_size, queryL, sourceL)
    # --> batch x sourceL x queryL
    attnT = torch.transpose(attn, 1, 2).contiguous()

    # (batch x ndf x sourceL)(batch x sourceL x queryL)
    # --> batch x ndf x queryL
    weightedContext = torch.bmm(context, attnT)

    return weightedContext, attn.view(batch_size, -1, ih, iw)

class CNN_ENCODER(nn.Module):
    def __init__(self, nef):
        super(CNN_ENCODER, self).__init__()
        if True:
            self.nef = nef
        else:
            self.nef = 256  # define a uniform ranker

        #model = models.inception_v3()
        #url = 'https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth'
        # model.load_state_dict(model_zoo.load_url(url))
        model = models.inception_v3(pretrained=True, transform_input=False)
        for param in model.parameters():
            param.requires_grad = False
        #print('Load pretrained model from ', url)
        # print(model)

        print('Load pretrained inception v3 model')
        self.define_module(model)
        self.init_trainable_weights()

    def define_module(self, model):
        self.Conv2d_1a_3x3 = model.Conv2d_1a_3x3
        self.Conv2d_2a_3x3 = model.Conv2d_2a_3x3
        self.Conv2d_2b_3x3 = model.Conv2d_2b_3x3
        self.Conv2d_3b_1x1 = model.Conv2d_3b_1x1
        self.Conv2d_4a_3x3 = model.Conv2d_4a_3x3
        self.Mixed_5b = model.Mixed_5b
        self.Mixed_5c = model.Mixed_5c
        self.Mixed_5d = model.Mixed_5d
        self.Mixed_6a = model.Mixed_6a
        self.Mixed_6b = model.Mixed_6b
        self.Mixed_6c = model.Mixed_6c
        self.Mixed_6d = model.Mixed_6d
        self.Mixed_6e = model.Mixed_6e
        self.Mixed_7a = model.Mixed_7a
        self.Mixed_7b = model.Mixed_7b
        self.Mixed_7c = model.Mixed_7c

        self.emb_features = conv1x1(768, self.nef)
        self.emb_cnn_code = nn.Linear(2048, self.nef)

    def init_trainable_weights(self):
        initrange = 0.1
        self.emb_features.weight.data.uniform_(-initrange, initrange)
        self.emb_cnn_code.weight.data.uniform_(-initrange, initrange)

    def forward(self, x):
        features = None
        # --> fixed-size input: batch x 3 x 299 x 299
        x = nn.functional.interpolate(x, size=(299, 299), mode='bilinear', align_corners=False)
        # 299 x 299 x 3
        x = self.Conv2d_1a_3x3(x)
        # 149 x 149 x 32
        x = self.Conv2d_2a_3x3(x)
        # 147 x 147 x 32
        x = self.Conv2d_2b_3x3(x)
        # 147 x 147 x 64
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        # 73 x 73 x 64
        x = self.Conv2d_3b_1x1(x)
        # 73 x 73 x 80
        x = self.Conv2d_4a_3x3(x)
        # 71 x 71 x 192

        x = F.max_pool2d(x, kernel_size=3, stride=2)
        # 35 x 35 x 192
        x = self.Mixed_5b(x)
        # 35 x 35 x 256
        x = self.Mixed_5c(x)
        # 35 x 35 x 288
        x = self.Mixed_5d(x)
        # 35 x 35 x 288

        x = self.Mixed_6a(x)
        # 17 x 17 x 768
        x = self.Mixed_6b(x)
        # 17 x 17 x 768
        x = self.Mixed_6c(x)
        # 17 x 17 x 768
        x = self.Mixed_6d(x)
        # 17 x 17 x 768
        x = self.Mixed_6e(x)
        # 17 x 17 x 768

        # image region features
        features = x
        # 17 x 17 x 768

        x = self.Mixed_7a(x)
        # 8 x 8 x 1280
        x = self.Mixed_7b(x)
        # 8 x 8 x 2048
        x = self.Mixed_7c(x)
        # 8 x 8 x 2048
        x = F.avg_pool2d(x, kernel_size=8)
        # 1 x 1 x 2048
        # x = F.dropout(x, training=self.training)
        # 1 x 1 x 2048
        x = x.view(x.size(0), -1)
        # 2048

        # global image features
        cnn_code = self.emb_cnn_code(x)
        # 512
        if features is not None:
            features = self.emb_features(features)
        return features, cnn_code

<h3>Calculate the cosine similarity, this function will be used in the evaluation </h3>

In [ ]:
def cosine_similarity(x1, x2, dim=1, eps=1e-8):
    """Returns cosine similarity between x1 and x2, computed along dim.
    """
    w12 = torch.sum(x1 * x2, dim)
    w1 = torch.norm(x1, 2, dim)
    w2 = torch.norm(x2, 2, dim)
    return (w12 / (w1 * w2).clamp(min=eps)).squeeze()


def words_loss(img_features, words_emb, labels,
               cap_lens, class_ids, batch_size):
    """
        words_emb(query): batch x nef x seq_len
        img_features(context): batch x nef x 17 x 17
    """
    masks = []
    att_maps = []
    similarities = []
    cap_lens = cap_lens.data.tolist()
    for i in range(batch_size):
        if class_ids is not None:
            mask = (class_ids == class_ids[i])
            mask[i] = 0
            masks.append(mask.view(1, -1))
        # Get the i-th text description
        words_num = cap_lens[i]
        # -> 1 x nef x words_num
        word = words_emb[i, :, :words_num].unsqueeze(0).contiguous()
        # -> batch_size x nef x words_num
        word = word.repeat(batch_size, 1, 1)
        # batch x nef x 17*17
        context = img_features
        """
            word(query): batch x nef x words_num
            context: batch x nef x 17 x 17
            weiContext: batch x nef x words_num
            attn: batch x words_num x 17 x 17
        """
        weiContext, attn = func_attention(word.cuda(), context.cuda(), 5.0)
        att_maps.append(attn[i].unsqueeze(0).contiguous())
        # --> batch_size x words_num x nef
        word = word.transpose(1, 2).contiguous()
        weiContext = weiContext.transpose(1, 2).contiguous()
        # --> batch_size*words_num x nef
        word = word.view(batch_size * words_num, -1)
        weiContext = weiContext.view(batch_size * words_num, -1)
        #
        # -->batch_size*words_num
        row_sim = cosine_similarity(word, weiContext)
        # --> batch_size x words_num
        row_sim = row_sim.view(batch_size, words_num)

        # Eq. (10)
        row_sim.mul_(5.0).exp_()
        row_sim = row_sim.sum(dim=1, keepdim=True)
        row_sim = torch.log(row_sim)

        # --> 1 x batch_size
        # similarities(i, j): the similarity between the i-th image and the j-th text description
        similarities.append(row_sim)

    # batch_size x batch_size
    similarities = torch.cat(similarities, 1)
    if class_ids is not None:
        # masks: batch_size x batch_size
        masks = torch.cat(masks, dim=0)
        masks = masks.cuda()
    similarities = similarities * 10.0
    if class_ids is not None:
        similarities.data.masked_fill_(masks, -float('inf'))
    similarities1 = similarities.transpose(0, 1)
    if labels is not None:
        loss0 = nn.CrossEntropyLoss()(similarities, labels)
        loss1 = nn.CrossEntropyLoss()(similarities1, labels)
    else:
        loss0, loss1 = None, None
    return loss0, loss1, att_maps


# ##################Loss for G and Ds##############################



def DAMSM_loss(image_encoder, fake_imgs, real_labels,
               words_embs, sent_emb, match_labels,
               cap_lens, class_ids):
    class_ids = torch.LongTensor(class_ids)
    batch_size = real_labels.size(0)
    # Forward

    # words_features: batch_size x nef x 17 x 17
    # sent_code: batch_size x nef
    region_features, cnn_code = image_encoder(fake_imgs)
    w_loss0, w_loss1, _ = words_loss(region_features, words_embs,
                                     match_labels, cap_lens,
                                     class_ids, batch_size)
    w_loss = (w_loss0 + w_loss1) * \
        1.0
    # err_words = err_words + w_loss.data[0]

    # s_loss0, s_loss1 = sent_loss(cnn_code, sent_emb,
    #                             match_labels, class_ids, batch_size)
    # s_loss = (s_loss0 + s_loss1) * \
    #    cfg.TRAIN.SMOOTH.LAMBDA
    # err_sent = err_sent + s_loss.data[0]

    #DAMSM = w_loss + s_loss

    DAMSM = w_loss
    return DAMSM

<h3>Define the model, you don't have to upload some pre-trained model in the first, I upload them because a break dowen the training process</h3>

In [ ]:
image_encoder = CNN_ENCODER(256)
#image_encoder_optimazer = torch.optim.Adam(image_encoder.parameters(), lr=0.00004, betas=(0.0, 0.9))
#image_encoder.cuda()
encoder_parameters = list(text_encoder.parameters())
for v in image_encoder.parameters():
    encoder_parameters.append(v)
optimizerEncoder = torch.optim.Adam(encoder_parameters, lr=0.00004, betas=(0.0, 0.9))
image_encoder.cuda()
text_encoder.cuda()


In [ ]:
### device = "cuda"
import torchvision.utils as vutils
import random
batch_size = 32
random.seed(100)
np.random.seed(100)
torch.manual_seed(100)
torch.cuda.manual_seed_all(100)
CUDA_LAUNCH_BLOCKING=1
netG = NetG(batch_size, 100).to(device)
netD = NetD(batch_size).to(device)
optimizerG = torch.optim.Adam(netG.parameters(), lr=0.0001, betas=(0.0, 0.9))
optimizerD = torch.optim.Adam(netD.parameters(), lr=0.0004, betas=(0.0, 0.9))
checkpoint_G = torch.load('../input/f-modul-ssa-dam-24/netG_Flower_vSSA_DAMSM_30.pt')
netG.load_state_dict(checkpoint_G['model_state_dict'])
optimizerG.load_state_dict(checkpoint_G['optimizer_state_dict'])
optimizerEncoder.load_state_dict(checkpoint_G['optimizerEncoder'])
text_encoder.load_state_dict(checkpoint_G['text_encoder'])
image_encoder.load_state_dict(checkpoint_G['image_encoder'])
epoch_ = checkpoint_G['epoch']

checkpoint_D = torch.load('../input/f-modul-ssa-dam-24/netD_Flower_vSSA_DAMSM_30.pt')
netD.load_state_dict(checkpoint_D['model_state_dict'])
optimizerD.load_state_dict(checkpoint_D['optimizer_state_dict'])
netG.train()
netD.train()
image_encoder.train()
text_encoder.train()

<h3>The training loop, and the save of the model each 2 epochs.</h3>

In [ ]:

for epoch in range(epoch_ +1,epoch_ +41):
    i = 0
    for images,captions,cap_lens,class_ids in train_loader:
        cap_lens,index = torch.sort(cap_lens,descending = True)
        captions = captions[index]
        images = images[index]
        class_ids = class_ids[index]
        real_labels, fake_labels, match_labels = prepare_labels(batch_size)
        hidden = text_encoder.init_hidden(batch_size)
            # words_embs: batch_size x nef x seq_len
            # sent_emb: batch_size x nef
        words_embs, sent_emb = text_encoder(captions.to(device), cap_lens.to(device), hidden)
        words_embs_de, sent_emb_de = words_embs.detach().to(device), sent_emb.detach().to(device)
        imgs = images.to(device)
        real_features = netD(imgs.to(device))
        output = netD.COND_DNET(real_features.to(device), sent_emb_de.to(device))
        errD_real = torch.nn.ReLU()(1.0 - output).mean()

        output = netD.COND_DNET(real_features[:(batch_size - 1)].to(device), sent_emb_de[1:batch_size].to(device))
        errD_mismatch = torch.nn.ReLU()(1.0 + output).mean()

            # synthesize fake images
        noise = torch.randn(batch_size, 100)
        noise = noise.to(device)
        fake, _ = netG(noise, sent_emb_de.to(device))

            # update encoder
        DAMSM_D = DAMSM_loss(image_encoder, images.to(device), real_labels, words_embs.to(device),
                                 sent_emb.to(device), match_labels, cap_lens.to(device), class_ids)
        optimizerEncoder.zero_grad()
        DAMSM_D.backward()
        optimizerEncoder.step()

            # G does not need update with D
        fake_features = netD(fake.detach())

        errD_fake = netD.COND_DNET(fake_features.to(device), sent_emb_de.to(device))
        errD_fake = torch.nn.ReLU()(1.0 + errD_fake).mean()

        errD = errD_real + (errD_fake + errD_mismatch) / 2.0
        optimizerD.zero_grad()
        errD.backward()
        optimizerD.step()

            # MA-GP
        interpolated = (imgs.data).requires_grad_()
        sent_inter = (sent_emb_de.data).requires_grad_()
        features = netD(interpolated.to(device))
        out = netD.COND_DNET(features, sent_inter)
        grads = torch.autograd.grad(outputs=out,
                                        inputs=(interpolated, sent_inter),
                                        grad_outputs=torch.ones(out.size()).cuda(),
                                        retain_graph=True,
                                        create_graph=True,
                                        only_inputs=True)
        grad0 = grads[0].view(grads[0].size(0), -1).to(device)
        grad1 = grads[1].view(grads[1].size(0), -1).to(device)
        grad = torch.cat((grad0, grad1), dim=1)
        grad_l2norm = torch.sqrt(torch.sum(grad ** 2, dim=1))
        d_loss_gp = torch.mean((grad_l2norm) ** 6)
        d_loss = 2.0 * d_loss_gp
        optimizerD.zero_grad()
        d_loss.backward()
        optimizerD.step()

            # update G
        features = netD(fake)
        output = netD.COND_DNET(features.to(device), sent_emb_de.to(device))
        errG = - output.mean()
        DAMSM_G = 0.1 * DAMSM_loss(image_encoder, fake.to(device), real_labels, words_embs_de.to(device),
                                       sent_emb_de.to(device), match_labels, cap_lens.to(device), class_ids)
        errG_total = errG + DAMSM_G
        optimizerG.zero_grad()
        errG_total.backward()
        optimizerG.step()
        print('[%d/%d][%d/%d] Loss_D: %.3f Loss_G %.3f DAMSM_D %.3f DAMSM_G %.3f'
                % (epoch, 600, i, len(train_loader), errD.item(), errG.item(),DAMSM_D.item(),DAMSM_G.item()))
        i +=1
        if epoch%2 == 0 and i == 224:
            print("save ................")
            vutils.save_image(fake.data,
                        '%s/fake_samples_epoch_%03d.png' % ('./', epoch+1),
                        normalize=True)
            torch.save({
            'model_state_dict': netD.state_dict(),
            'optimizer_state_dict': optimizerD.state_dict(),
            },'./netD_Flower_vSSA_DAMSM_54.pt')
            
            torch.save({
            'epoch': epoch,
            'model_state_dict': netG.state_dict(),
            'optimizer_state_dict': optimizerG.state_dict(),
            'text_encoder':text_encoder.state_dict(),
            'optimizerEncoder': optimizerEncoder.state_dict(),
            'image_encoder':image_encoder.state_dict(),
            },'./netG_Flower_vSSA_DAMSM_54.pt')

In [ ]:
            print("save ................")
            vutils.save_image(fake.data,
                        '%s/fake_samples_epoch_%03d.png' % ('./', epoch+1),
                        normalize=True)
            torch.save({
            'model_state_dict': netD.state_dict(),
            'optimizer_state_dict': optimizerD.state_dict(),
            },'./netD_Flower_vSSA_DAMSM_30.pt')
            
            torch.save({
            'epoch': epoch,
            'model_state_dict': netG.state_dict(),
            'optimizer_state_dict': optimizerG.state_dict(),
            'text_encoder':text_encoder.state_dict(),
            'optimizerEncoder': optimizerEncoder.state_dict(),
            'image_encoder':image_encoder.state_dict(),
            },'./netG_Flower_vSSA_DAMSM_30.pt')

##### os.mkdir('./Output/')

<h3>The evaluation of the model using the test set</h3>

In [ ]:
#os.mkdir('./Output/')
batch_size = 32
save_dir = './Output/'
d = 0
t = 0
for i in range(1):  # (cfg.TEXT.CAPTIONS_PER_IMAGE):
    for images,captions,cap_lens in test_loader:
        cap_lens,index = torch.sort(cap_lens,descending = True)
        captions = captions[index]
        images = images[index]
        with torch.no_grad():
            hidden = text_encoder.init_hidden(batch_size)
            words_embs, sent_emb = text_encoder(captions.to(device), cap_lens.to(device), hidden)
            words_embs, sent_emb = words_embs.detach().to(device), sent_emb.detach().to(device)
            #sent_emb = sent_emb.transpose(0, 1).contiguous()
            
        #print("captions :",captions.size())
        #print("captions :",captions[3])
        #print("images :",images.size())
        #words_embs, sent_emb = text_encoder(captions.to(device),cap_lens.to(device),hidden)
        #words_embs, sent_emb = words_embs.detach().to(device), sent_emb.detach().to(device)
            #######################################################
            # (2) Generate fake images
            ######################################################
        with torch.no_grad():
            noise = torch.randn(batch_size, 100)
            noise=noise.to(device)
            fake_imgs,_ = netG(noise,sent_emb)
        print(">>>>>>>>batch : ",d)
        d += 1
        for j in range(batch_size):
            s_tmp = './Output/'
            im = fake_imgs[j].data.cpu().numpy()
                # [-1, 1] --> [0, 255]
            #arab_cap = captions[j]
            im = (im + 1.0) * 127.5
            im = im.astype(np.uint8)
            im = np.transpose(im, (1, 2, 0))
            im = Image.fromarray(im)
            #arab_cap = tokenizer.decode(arab_cap)
            #cap_path = '%s_%3d.txt' % (s_tmp,t)
            fullpath = '%s_%3d.png' % (s_tmp,t)
            t += 1
            im.save(fullpath)
           # with open(cap_path,'w') as f:
           #     f.write(arab_cap)
                

In [ ]:
!zip -r Output.zip ./Output

<h3>The use of FID and IS to evaluate the Quality of the generated images</h3>

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models


class InceptionV3(nn.Module):
    """Pretrained InceptionV3 network returning feature maps"""

    # Index of default block of inception to return,
    # corresponds to output of final average pooling
    DEFAULT_BLOCK_INDEX = 3

    # Maps feature dimensionality to their output blocks indices
    BLOCK_INDEX_BY_DIM = {
        64: 0,   # First max pooling features
        192: 1,  # Second max pooling featurs
        768: 2,  # Pre-aux classifier features
        2048: 3  # Final average pooling features
    }

    def __init__(self,
                 output_blocks=[DEFAULT_BLOCK_INDEX],
                 resize_input=True,
                 normalize_input=True,
                 requires_grad=False):
        """Build pretrained InceptionV3
        Parameters
        ----------
        output_blocks : list of int
            Indices of blocks to return features of. Possible values are:
                - 0: corresponds to output of first max pooling
                - 1: corresponds to output of second max pooling
                - 2: corresponds to output which is fed to aux classifier
                - 3: corresponds to output of final average pooling
        resize_input : bool
            If true, bilinearly resizes input to width and height 299 before
            feeding input to model. As the network without fully connected
            layers is fully convolutional, it should be able to handle inputs
            of arbitrary size, so resizing might not be strictly needed
        normalize_input : bool
            If true, normalizes the input to the statistics the pretrained
            Inception network expects
        requires_grad : bool
            If true, parameters of the model require gradient. Possibly useful
            for finetuning the network
        """
        super(InceptionV3, self).__init__()

        self.resize_input = resize_input
        self.normalize_input = normalize_input
        self.output_blocks = sorted(output_blocks)
        self.last_needed_block = max(output_blocks)

        assert self.last_needed_block <= 3, \
            'Last possible output block index is 3'

        self.blocks = nn.ModuleList()

        inception = models.inception_v3(pretrained=True)

        # Block 0: input to maxpool1
        block0 = [
            inception.Conv2d_1a_3x3,
            inception.Conv2d_2a_3x3,
            inception.Conv2d_2b_3x3,
            nn.MaxPool2d(kernel_size=3, stride=2)
        ]
        self.blocks.append(nn.Sequential(*block0))

        # Block 1: maxpool1 to maxpool2
        if self.last_needed_block >= 1:
            block1 = [
                inception.Conv2d_3b_1x1,
                inception.Conv2d_4a_3x3,
                nn.MaxPool2d(kernel_size=3, stride=2)
            ]
            self.blocks.append(nn.Sequential(*block1))

        # Block 2: maxpool2 to aux classifier
        if self.last_needed_block >= 2:
            block2 = [
                inception.Mixed_5b,
                inception.Mixed_5c,
                inception.Mixed_5d,
                inception.Mixed_6a,
                inception.Mixed_6b,
                inception.Mixed_6c,
                inception.Mixed_6d,
                inception.Mixed_6e,
            ]
            self.blocks.append(nn.Sequential(*block2))

        # Block 3: aux classifier to final avgpool
        if self.last_needed_block >= 3:
            block3 = [
                inception.Mixed_7a,
                inception.Mixed_7b,
                inception.Mixed_7c,
                nn.AdaptiveAvgPool2d(output_size=(1, 1))
            ]
            self.blocks.append(nn.Sequential(*block3))

        for param in self.parameters():
            param.requires_grad = requires_grad

    def forward(self, inp):
        """Get Inception feature maps
        Parameters
        ----------
        inp : torch.autograd.Variable
            Input tensor of shape Bx3xHxW. Values are expected to be in 
            range (0, 1)
        Returns
        -------
        List of torch.autograd.Variable, corresponding to the selected output 
        block, sorted ascending by index
        """
        outp = []
        x = inp

        if self.resize_input:
            x = F.upsample(x, size=(299, 299), mode='bilinear', align_corners=True)

        if self.normalize_input:
            x = x.clone()
            x[:, 0] = x[:, 0] * (0.229 / 0.5) + (0.485 - 0.5) / 0.5
            x[:, 1] = x[:, 1] * (0.224 / 0.5) + (0.456 - 0.5) / 0.5
            x[:, 2] = x[:, 2] * (0.225 / 0.5) + (0.406 - 0.5) / 0.5

        for idx, block in enumerate(self.blocks):
            x = block(x)
            if idx in self.output_blocks:
                outp.append(x)

            if idx == self.last_needed_block:
                break

        return outp

In [ ]:
block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[2048]
model_incp = InceptionV3([block_idx])
model_incp=model_incp.cuda()

In [ ]:
def calculate_activation_statistics(images,model,batch_size=128, dims=2048,
                    cuda=False):
    model.eval()
    act=np.empty((len(images), dims))
    
    if cuda:
        batch=images.cuda()
    else:
        batch=images
    pred = model(batch)[0]

        # If model output is not scalar, apply global spatial average pooling.
        # This happens if you choose a dimensionality not equal 2048.
    if pred.size(2) != 1 or pred.size(3) != 1:
        pred = adaptive_avg_pool2d(pred, output_size=(1, 1))

    act= pred.cpu().data.numpy().reshape(pred.size(0), -1)
    
    mu = np.mean(act, axis=0)
    sigma = np.cov(act, rowvar=False)
    return mu, sigma

In [ ]:
def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """Numpy implementation of the Frechet Distance.
    The Frechet distance between two multivariate Gaussians X_1 ~ N(mu_1, C_1)
    and X_2 ~ N(mu_2, C_2) is
            d^2 = ||mu_1 - mu_2||^2 + Tr(C_1 + C_2 - 2*sqrt(C_1*C_2)).
    """

    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, \
        'Training and test mean vectors have different lengths'
    assert sigma1.shape == sigma2.shape, \
        'Training and test covariances have different dimensions'

    diff = mu1 - mu2

    
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        msg = ('fid calculation produces singular product; '
               'adding %s to diagonal of cov estimates') % eps
        print(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError('Imaginary component {}'.format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return (diff.dot(diff) + np.trace(sigma1) +
            np.trace(sigma2) - 2 * tr_covmean)

In [ ]:
def calculate_fretchet(images_real,images_fake,model):
    mu_1,std_1=calculate_activation_statistics(images_real,model,cuda=True)
    mu_2,std_2=calculate_activation_statistics(images_fake,model,cuda=True)
    fid_value = calculate_frechet_distance(mu_1, std_1, mu_2, std_2)
    return fid_value

In [ ]:
path = "./Output/"
output_images = []
with os.scandir('./Output') as entrie:
    for entry in entrie:
        output_images.append(path+entry.name)

In [ ]:
def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')
    
        
train_transforms =  transforms.Compose([
                                 transforms.Scale(32),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                             ])

class dataset(torch.utils.data.Dataset):

    def __init__(self,images_paths,transform=None):
        self.images_paths = images_paths 
        self.transform = transform 
         
    #dataset length
    def __len__(self):
        self.filelength = len(self.images_paths)
        return self.filelength
    
    #load an one of images
    def __getitem__(self,idx):
     #   img_path = self.file_list[idx]
        img = pil_loader(self.images_paths[idx].strip())
        img_transformed = self.transform(img)   
        #img_transformed = img_transformed.expand(3,256,256)
        return img_transformed

In [ ]:
generated_images = dataset(output_images,transform=train_transforms)

In [ ]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data

from torchvision.models.inception import inception_v3

import numpy as np
from scipy.stats import entropy

def inception_score(imgs, cuda=True, batch_size=32, resize=True, splits=1):
    """Computes the inception score of the generated images imgs
    imgs -- Torch dataset of (3xHxW) numpy images normalized in the range [-1, 1]
    cuda -- whether or not to run on GPU
    batch_size -- batch size for feeding into Inception v3
    splits -- number of splits
    """
    N = len(imgs)

    assert batch_size > 0
    assert N > batch_size

    # Set up dtype
    if cuda:
        dtype = torch.cuda.FloatTensor
    else:
        if torch.cuda.is_available():
            print("WARNING: You have a CUDA device, so you should probably set cuda=True")
        dtype = torch.FloatTensor

    # Set up dataloader
    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)

    # Load inception model
    inception_model = inception_v3(pretrained=True, transform_input=False).type(dtype)
    inception_model.eval();
    up = nn.Upsample(size=(299, 299), mode='bilinear').type(dtype)
    def get_pred(x):
        if resize:
            x = up(x)
        x = inception_model(x)
        return F.softmax(x).data.cpu().numpy()

    # Get predictions
    preds = np.zeros((N, 1000))

    for i, batch in enumerate(dataloader, 0):
        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)

    # Now compute the mean kl-div
    split_scores = []

    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))

    return np.mean(split_scores), np.std(split_scores)


import torchvision.datasets as dset
import torchvision.transforms as transforms

print ("Calculating Inception Score...")
print (inception_score(generated_images, cuda=True, batch_size=32, resize=True, splits=10))

In [ ]:
from scipy import linalg
from PIL import Image
batch_size = 32
save_dir = './Output/'
frech = []
d = 0
t = 0
    
for i in range(1):  # (cfg.TEXT.CAPTIONS_PER_IMAGE):
    for images,captions,cap_lens in test_loader:
        cap_lens,index = torch.sort(cap_lens,descending = True)
        captions = captions[index]
        images = images[index]
        with torch.no_grad():
            hidden = text_encoder.init_hidden(batch_size)
            words_embs, sent_emb = text_encoder(captions.to(device), cap_lens.to(device), hidden)
            words_embs, sent_emb = words_embs.detach().to(device), sent_emb.detach().to(device)
            #sent_emb = sent_emb.transpose(0, 1).contiguous()
            
        #print("captions :",captions.size())
        #print("captions :",captions[3])
        #print("images :",images.size())
        #words_embs, sent_emb = text_encoder(captions.to(device),cap_lens.to(device),hidden)
        #words_embs, sent_emb = words_embs.detach().to(device), sent_emb.detach().to(device)
            #######################################################
            # (2) Generate fake images
            ######################################################
        with torch.no_grad():
            noise = torch.randn(batch_size, 100)
            noise=noise.to(device)
            fake_imgs,_ = netG(noise,sent_emb)
        fd = calculate_fretchet(images,fake_imgs,model_incp)
        print(fd)
        frech.append(fd)
        
        

In [ ]:
!zip -r ./output.zip ./Output